In [57]:
import numpy as np
from thewalrus.random import random_symplectic, randnc
# import seaborn as sns
import matplotlib.pyplot as plt

In [87]:
class GaussianState:
    """
    Represents an m-mode Gaussian state. [cite: 239]
    
    Attributes:
        m (int): The number of modes. [cite: 237, 238]
        mean (np.ndarray): First-moment vector in R^{2m}. 
        cov (np.ndarray): Covariance matrix in R^{2m x 2m}. 
    """
    def __init__(self, mean, cov):
        # Ensure the mean is a 1D array and the covariance is a 2D array
        self.mean = np.array(mean, dtype=np.float64).flatten()
        self.cov = np.array(cov, dtype=np.float64)
        
        # Determine dimension and number of modes
        # The phase space dimension is 2m [cite: 242, 247]
        self.dim = len(self.mean)
        if self.dim % 2 != 0:
            raise ValueError("The mean vector must have an even dimension (2m).")
            
        if self.cov.shape != (self.dim, self.dim):
            raise ValueError(f"Covariance matrix must be {self.dim}x{self.dim}.")
            
        self.m = self.dim // 2 

    def __repr__(self):
        return f"GaussianState(modes={self.m}, mean_norm={self.mean})"

    @classmethod
    def coherent(cls, m_modes, mean_vector=None):
        """
        Creates an m-mode coherent state.
        
        Args:
            m_modes (int): Number of modes for the state 
            mean_vector (np.ndarray, optional): A 2m-dimensional vector 
                representing the displacement (x1, p1, ..., xm, pm). 
                If None, creates a vacuum state.
        """
        dim = 2 * m_modes
        
        # If no mean vector is provided, default to the vacuum state (zero mean) 
        if mean_vector is None:
            mean = np.zeros(dim)
        else:
            mean = np.array(mean_vector, dtype=np.float64)
            if len(mean) != dim:
                raise ValueError(f"Mean vector must have length {dim} for {m_modes} modes.")
        
        # Coherent states always have a covariance matrix equal to the identity 
        cov = np.eye(dim)
        
        return cls(mean, cov)
        
    def mean_photon_number(self):
        """
        Computes the mean photon number (energy) of the Gaussian state.
        Based on Eq. (2.23) from the paper.
        """
        # Term 1: (Tr(V) - Tr(I)) / 4
        # Note: Tr(I) for a 2m x 2m matrix is simply the dimension (2m)
        trace_term = (np.trace(self.cov) - self.dim) / 4.0
        
        # Term 2: ||m||^2 / 2
        # This is the squared Euclidean norm of the mean vector divided by 2
        displacement_term = np.linalg.norm(self.mean)**2 / 2.0
        
        return trace_term + displacement_term

    def measure_heterodyne(self, num_meas=1):
        """
        Simulates heterodyne measurement on the Gaussian state.
        
        Args:
            num_meas (int): The number of independent measurement samples to generate.
            
        Returns:
            np.ndarray: An array of shape (num_meas, 2m) containing the classical 
                        outcomes for each measurement.
        """
        # Define the heterodyne covariance matrix: (V + I) / 2
        het_cov = (self.cov + np.eye(self.dim)) / 2.0
        
        # Generate num_meas samples from the multivariate normal distribution
        # defined by the state's mean and the heterodyne covariance.
        samples = np.random.multivariate_normal(
            mean=self.mean, 
            cov=het_cov, 
            size=num_meas
        )
        
        return samples

In [86]:
class GaussianUnitary:
    """
    Represents a bosonic Gaussian unitary G = Dr * Us.
    
    Attributes:
        m (int): Number of modes.
        dim (int): Phase-space dimension (2 * m).
        S (np.ndarray): 2m x 2m real symplectic matrix.
        r (np.ndarray): 2m-dimensional real displacement vector.
    """
    def __init__(self, S, r):
        # The phase-space dimension is determined by the size of the symplectic matrix 
        self.dim = S.shape[0]
        
        # Verify that the matrix is square and the displacement vector matches its size
        if S.shape[0] != S.shape[1]:
            raise ValueError("Symplectic matrix S must be square.")
        if len(r) != self.dim:
            raise ValueError(f"Displacement vector r must have dimension {self.dim}.")
        if self.dim % 2 != 0:
            raise ValueError("Phase-space dimension must be even (2 * m).")
            
        self.S = np.array(S, dtype=np.float64)
        self.r = np.array(r, dtype=np.float64)
        self.m = self.dim // 2

    def __repr__(self):
        return f"GaussianUnitary(modes={self.m}, dim={self.dim})"

    @classmethod
    def random(cls, m, z):
        """
        Creates a random Gaussian unitary object defined by (S, r) such that z is the scale parameter on a random complex number (need to fix this to have more transparent meaning of z).
        """
        # Get random symplectic
        S = random_symplectic(m, scale=np.log(z)/np.sqrt(2))
        
        # Create a random displacement vector r [cite: 248]
        # We sample r from a standard normal distribution for convenience
        r = np.random.normal(0, 1, 2*m)
        
        return cls(S, r)

    def evolve(self, state):
        """
        Applies this Gaussian unitary to a GaussianState object.
        Returns a new GaussianState with updated mean and covariance.
        """
        if state.m != self.m:
            raise ValueError(f"State modes ({state.m}) must match unitary modes ({self.m}).")

        # 1. Update the first moment (mean): m' = Sm + r
        # Based on Eq. (2.9) and (2.11) in the paper
        new_mean = self.S @ state.mean + self.r

        # 2. Update the second moment (covariance): V' = S V S^T
        # Based on Eq. (2.10) and (2.12) in the paper
        new_cov = self.S @ state.cov @ self.S.T

        return GaussianState(new_mean, new_cov)

In [76]:
# Example usage:
# m = 3 modes, squeezing bound z = 2.0
z = 1
G = GaussianUnitary.random(2, z)

print(f"Symplectic Matrix S (shape {S.shape}):\n{G.S}")
print(f"\nDisplacement Vector r:\n{G.r}")
np.linalg.norm(G.S, ord=np.inf)

Symplectic Matrix S (shape (4, 4)):
[[ 0.22025394  0.01617094  0.27013982 -0.93715056]
 [ 0.10485471  0.25326118 -0.93140655 -0.23947045]
 [-0.27013982  0.93715056  0.22025394  0.01617094]
 [ 0.93140655  0.23947045  0.10485471  0.25326118]]

Displacement Vector r:
[-1.12569077 -0.35350011  0.81898076  0.3975143 ]


np.float64(1.5289928923855518)